# Exploring Descriptive and Inferential Statistics Using Python for Data-Driven Decision-Making

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats
from scipy.stats import norm
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [ ]:
df = pd.read_csv("C:/Users/Vostro/OneDrive/Desktop/MSc Data Science/Semester 1/Research Project/online_retail.csv", encoding='ISO-8859-1')

In [ ]:
df.info()

In [ ]:
df.head(10)

In [ ]:
df.tail(10)

In [ ]:
df.columns

In [ ]:
df.describe()

In [ ]:
# Check for missing values
missing_values = df.isnull().sum()
missing_percent = (missing_values / len(df)) * 100
print("Missing Values:\n", missing_values)
print("Missing Values Percentage:\n", missing_percent)

In [ ]:
# 1. Data Preprocessing


# Drop rows with missing values in both 'CustomerID' and 'Description'
df.dropna(subset=['CustomerID','Description'], inplace=True)

# Remove negative or zero quantities and unit prices
df = df[df['Quantity'] > 0]
df = df[df['UnitPrice'] > 0]
print(df.isnull().sum())

In [ ]:
#Removing outleirs which can affecting the analysis

def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return df[(df[column] >= lower) & (df[column] <= upper)]

# Remove outliers from both Quantity and UnitPrice
df_clean = remove_outliers_iqr(df, 'Quantity')
df_clean = remove_outliers_iqr(df_clean, 'UnitPrice')

In [ ]:
# Before Removing Outliers
fig_before = px.box(df, y=['Quantity', 'UnitPrice'], title="Before Removing Outliers")
fig_before.show()

# After Removing Outliers
fig_after = px.box(df_clean, y=['Quantity', 'UnitPrice'], title="After Removing Outliers")
fig_after.show()

In [ ]:
# Convert InvoiceDate to datetime
df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'])

In [ ]:
df_clean.info()

In [ ]:
# 2. Descriptive Statistics

# Central tendency & dispersion

df_clean[['Quantity', 'UnitPrice']].describe()

In [ ]:
# 3. Exploratory Data Analysis (EDA)

# Histogram for UnitPrice

fig1 = px.histogram(df_clean, x="UnitPrice", nbins=100, title="Distribution of Unit Price",
                    marginal="box", color_discrete_sequence=["indigo"])
fig1.update_layout(bargap=0.1)
fig1.show()

In [ ]:
# Boxplot by Country for Quantity

top_countries = df_clean['Country'].value_counts().nlargest(5).index
df_box = (df_clean[df_clean['Country'].isin(top_countries)])
fig2 = px.box(df_box, x="Country", y="Quantity", title="Quantity Distribution by Top 5 Countries",
              color="Country", color_discrete_sequence=px.colors.qualitative.Set1)
fig2.show()

In [ ]:
# Heatmap of correlation

corr = df_clean[['Quantity', 'UnitPrice']].corr()
fig3 = go.Figure(data=go.Heatmap(z=corr.values, x=corr.columns, y=corr.columns,
                                 colorscale='RdBu', zmin=-1, zmax=1))
fig3.update_layout(title="Correlation Heatmap")
fig3.show()

In [ ]:
# 4 Inferential Statistics 
# Multiple Linear Regression 

# Filter for Top 3 Countries
df_filtered = df_clean[df_clean['Country'].isin(['United Kingdom', 'Germany', 'France'])].copy()
df_filtered['InvoiceMonth'] = df_filtered['InvoiceDate'].dt.month

#Prepare Features & Encode Country
X = df_filtered[['UnitPrice', 'InvoiceMonth', 'Country']]
y = df_filtered['Quantity']

encoder = OneHotEncoder(categories=[['United Kingdom', 'France', 'Germany']],drop='first', sparse_output=False)
X_country = encoder.fit_transform(X[['Country']])
X_encoded = pd.DataFrame(X_country, columns=encoder.get_feature_names_out(['Country']))
X_final = pd.concat([X[['UnitPrice', 'InvoiceMonth']].reset_index(drop=True), X_encoded], axis=1)

#Train the Model
X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

#Evaluate
r2 = r2_score(y_test, y_pred)
print("R-squared:", r2)
print("Intercept:", model.intercept_)
print("Coefficients:", dict(zip(X_final.columns, model.coef_)))

In [ ]:
#Visualize Actual vs Predicted
fig = go.Figure()
fig.add_trace(go.Scatter(x=y_test[:500], y=y_pred[:500], mode='markers', name='Predicted vs Actual'))
fig.add_trace(go.Scatter(x=[0, max(y_test[:500])], y=[0, max(y_test[:500])], mode='lines',
                      line=dict(color='red', dash='dash'), name='Ideal Fit'))
fig.update_layout(title='Actual vs Predicted Quantity',
                  xaxis_title='Actual Quantity', yaxis_title='Predicted Quantity')


In [ ]:
#Shapiro-Wilk Test
quantity_sample = df_clean['Quantity'].sample(500, random_state=1)
unitprice_sample = df_clean['UnitPrice'].sample(500, random_state=1)
stats.shapiro(quantity_sample), stats.shapiro(unitprice_sample)

In [ ]:
#Visualization of Shapiro-Wilk Test
# Sample the data
quantity_sample = df_clean['Quantity'].sample(500, random_state=1)
unitprice_sample = df_clean['UnitPrice'].sample(500, random_state=1)

# Create DataFrame for Quantity
q_df = pd.DataFrame({'Value': quantity_sample})
q_df['Variable'] = 'Quantity'

# Create DataFrame for UnitPrice
u_df = pd.DataFrame({'Value': unitprice_sample})
u_df['Variable'] = 'UnitPrice'

# Combine both for one chart if needed
combined_df = pd.concat([q_df, u_df], ignore_index=True)

# Plot Quantity distribution
fig_q = px.histogram(q_df, x='Value', nbins=30, marginal="box", opacity=0.7,
                     title="Shapiro-Wilk: Quantity Sample Distribution with KDE",
                     histnorm='density')
fig_q.add_scatter(x=np.linspace(q_df['Value'].min(), q_df['Value'].max(), 100),
                  y=norm.pdf(np.linspace(q_df['Value'].min(), q_df['Value'].max(), 100),
                             q_df['Value'].mean(), q_df['Value'].std()),
                  mode='lines', name='Normal Curve', line=dict(color='red'))
fig_q.show()

# Plot UnitPrice distribution
fig_u = px.histogram(u_df, x='Value', nbins=30, marginal="box", opacity=0.7,
                     title="Shapiro-Wilk: UnitPrice Sample Distribution with KDE",
                     histnorm='density')
fig_u.add_scatter(x=np.linspace(u_df['Value'].min(), u_df['Value'].max(), 100),
                  y=norm.pdf(np.linspace(u_df['Value'].min(), u_df['Value'].max(), 100),
                             u_df['Value'].mean(), u_df['Value'].std()),
                  mode='lines', name='Normal Curve', line=dict(color='red'))
fig_u.show()

In [ ]:
#t-Test
uk_prices = df_clean[df_clean['Country'] == 'United Kingdom']['UnitPrice'].sample(500, random_state=1)
germany_prices = df_clean[df_clean['Country'] == 'Germany']['UnitPrice'].sample(500, random_state=1)
stats.ttest_ind(uk_prices, germany_prices, equal_var=False)

In [ ]:
#t-Test Visualization
# Sample data
uk_prices = df_clean[df_clean['Country'] == 'United Kingdom']['UnitPrice'].sample(500, random_state=1)
germany_prices = df_clean[df_clean['Country'] == 'Germany']['UnitPrice'].sample(500, random_state=1)

# Combine into one DataFrame
ttest_df = pd.DataFrame({
    'UnitPrice': pd.concat([uk_prices, germany_prices], ignore_index=True),
    'Country': ['United Kingdom'] * 500 + ['Germany'] * 500
})

# Plotly box plot with strip overlay
fig = px.box(ttest_df, x='Country', y='UnitPrice', points="all", color='Country',
             title="T-Test: Comparison of UnitPrice between UK and Germany",
             color_discrete_map={'United Kingdom': 'skyblue', 'Germany': 'orange'})
fig.update_traces(jitter=0.3, marker=dict(size=4, opacity=0.5))
fig.show()

In [ ]:
#ANOVA test
anova_df = df_clean[df_clean['Country'].isin(['United Kingdom', 'Germany', 'France'])]
stats.f_oneway(
    anova_df[anova_df['Country'] == 'United Kingdom']['Quantity'],
    anova_df[anova_df['Country'] == 'Germany']['Quantity'],
    anova_df[anova_df['Country'] == 'France']['Quantity']
)

In [ ]:
#ANOVA test visualization

top_countries = df_clean['Country'].value_counts().nlargest(3).index
anova_df = df_clean[df_clean['Country'].isin(top_countries)]
fig = px.box(anova_df, x='Country', y='Quantity', color='Country',
             title="ANOVA: Quantity Distribution by Country",
             color_discrete_sequence=px.colors.qualitative.Set2)
fig.show()


In [ ]:
summary = """
Final Summary and Insights

This notebook presented a complete data-driven analysis using the Online Retail dataset, focusing on applying descriptive and inferential statistical techniques through Python for business decision-making.

Key Steps and Insights:

1. Data Preprocessing:
- Removed missing CustomerIDs and negative/zero Quantity or UnitPrice values.
- Handled duplicates and outliers using IQR filtering and boxplots.
- Converted InvoiceDate to datetime and extracted InvoiceMonth.

2. Descriptive Statistics:
- Computed mean, median, std, min, and max for Quantity and UnitPrice.
- Plotted histograms, boxplots, and heatmaps to explore data distributions and relationships.
- Found right-skewed distributions and weak correlation between UnitPrice and Quantity.

3. Inferential Statistics:
- Shapiro-Wilk test showed non-normal distribution for both variables.
- t-Test showed no significant price difference between UK and Germany.
- ANOVA confirmed significant differences in Quantity sold across top countries.

4. Regression Analysis:
- Linear Regression predicted Quantity using UnitPrice, Month, and Country.
- R² = 0.135 suggests limited predictive power.
- UnitPrice had a negative impact on Quantity, indicating price sensitivity.

Business Implications:
- Regional pricing strategies and inventory allocation should be data-informed.
- Model could be improved by including more features (e.g., product type, promotions).
- Python enables reproducible, scalable data pipelines that empower evidence-based decision making.

Conclusion:
This analysis framework bridges statistical theory and practical business use. With further enhancements, it can support forecasting, segmentation, and real-time analytics in retail.
"""

print(summary)
